In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [8]:
train = pd.read_csv('../data/data3_train.csv')
test = pd.read_csv('../data/data3_test.csv')

In [9]:
target_column = "shot_made_flag"  # y_column_name

In [16]:
def split_x_y(df, target):
    y = df.loc[:, [target]]
    x = df.drop([target], axis=1)
    return x, y

def split_regular_playoff(df, season, train=None, test=None, validation=None):

    # if(train + test + validation != 1):
    #     print("Error dataset split")
    #     return None

    df_validaiton = None

    df_season = df[df['season'] == season]
    isPlayoffs = df_season.playoffs == 1

    df_train = df_season[~isPlayoffs]
    df_test  = df_season[isPlayoffs]

    print(f"Size of training dataset {len(df_train)}" )
    print(f"Size of test dataset {len(df_test)}" )
    print(f"Size of train vs test ratio {len(df_train) / (len(df_train) + len(df_test)) }" )

    return df_train, df_test
#     train = split_x_y(df_train, 'shot_made_flag')
#     test = split_x_y(df_test, 'shot_made_flag')
#     return train[0], train[1], test[0], test[1]
#     return train, test

# x_train, y_train, x_test, y_test = split_regular_playoff(df, '2010-11')
# train, test = split_regular_playoff(df, '2010-11')

In [13]:
train.head()

,loc_x,loc_y,period,playoffs,shot_distance,shot_made_flag,shot_zone_basic,shot_zone_range,time_remaining,last_5_games_avg,...,shot_type_2PT Field Goal,shot_type_3PT Field Goal,shot_zone_area_Back Court(BC),shot_zone_area_Center(C),shot_zone_area_Left Side Center(LC),shot_zone_area_Left Side(L),shot_zone_area_Right Side Center(RC),shot_zone_area_Right Side(R),matchup_away,matchup_home
0,0,0,3,0,0,0.0,0,0,225,0.0,...,1,0,0,1,0,0,0,0,0,1
1,0,0,3,0,0,0.0,0,0,429,0.0,...,1,0,0,1,0,0,0,0,0,1
2,144,41,3,0,14,1.0,2,1,159,0.0,...,1,0,0,0,0,0,0,1,0,1
3,-161,132,3,0,20,0.0,2,2,49,0.0,...,1,0,0,0,1,0,0,0,0,1
4,34,82,4,0,8,0.0,1,1,596,0.0,...,1,0,0,1,0,0,0,0,0,1


## Split X & Y

In [17]:
x_train, y_train = split_x_y(train, target_column)
x_test, y_test = split_x_y(test, target_column)

In [19]:
reg = LinearRegression()
reg.fit(x_train, y_train)
reg.score(x_train, y_train)

0.2299316121604148

array([[1],
       [0],
       [1],
       ...,
       [1],
       [0],
       [1]])

In [45]:
y_train_pred = reg.predict(x_train)
y_train_pred = y_train_pred > 0.5
y_train_pred = np.multiply(y_train_pred, 1)
accuracy_score(y_train.values, y_train_pred)

0.7180059523809523

In [33]:
y_train.values

array([[0.],
       [0.],
       [1.],
       ...,
       [0.],
       [1.],
       [1.]])

In [28]:
y_train_pred

array([[0.85580686],
       [0.37605526],
       [0.75709992],
       ...,
       [0.79694107],
       [0.25813182],
       [0.59538838]])

In [22]:
train

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,period,playoffs,...,game_date,matchup,opponent,shot_id,time_remaining,last_5_games_avg,streak_before_shot,points_before_shot,fgp_before_shot,month
19848,Driving Layup Shot,Layup,584,21000003,34.0213,18,23,-118.2518,4,0,...,2010-10-26,home,HOU,16453,105,0.39,0,10,0.36,10
19849,Jump Shot,Jump Shot,538,21000003,33.9173,-79,127,-118.3488,4,0,...,2010-10-26,home,HOU,16452,259,0.39,0,10,0.38,10
19850,Jump Shot,Jump Shot,350,21000003,33.9363,-180,108,-118.4498,3,0,...,2010-10-26,home,HOU,16450,400,0.39,2,10,0.42,10
19851,Jump Shot,Jump Shot,332,21000003,33.8983,-127,146,-118.3968,3,0,...,2010-10-26,home,HOU,16449,493,0.39,1,8,0.36,10
19852,Jump Shot,Jump Shot,322,21000003,33.9623,-106,82,-118.3758,3,0,...,2010-10-26,home,HOU,16448,546,0.39,0,6,0.30,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21203,Jump Shot,Jump Shot,468,21001230,33.9943,-98,50,-118.3678,4,0,...,2011-04-13,away,SAC,18067,89,0.42,0,22,0.69,4
21204,Jump Shot,Jump Shot,475,21001230,33.9553,144,89,-118.1258,4,0,...,2011-04-13,away,SAC,18068,57,0.42,0,22,0.65,4
21205,Jump Shot,Jump Shot,480,21001230,33.9963,-119,48,-118.3888,4,0,...,2011-04-13,away,SAC,18069,15,0.42,0,22,0.61,4
21206,Jump Shot,Jump Shot,506,21001230,33.9153,92,129,-118.1778,5,0,...,2011-04-13,away,SAC,18071,244,0.42,1,25,0.60,4


In [23]:
target_encoding(train, "action_type", target_column)

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,period,playoffs,...,matchup,opponent,shot_id,time_remaining,last_5_games_avg,streak_before_shot,points_before_shot,fgp_before_shot,month,action_type_te
0,Driving Layup Shot,Layup,584,21000003,34.0213,18,23,-118.2518,4,0,...,home,HOU,16453,105,0.39,0,10,0.36,10,0.706897
1,Jump Shot,Jump Shot,538,21000003,33.9173,-79,127,-118.3488,4,0,...,home,HOU,16452,259,0.39,0,10,0.38,10,0.326633
2,Jump Shot,Jump Shot,350,21000003,33.9363,-180,108,-118.4498,3,0,...,home,HOU,16450,400,0.39,2,10,0.42,10,0.326633
3,Jump Shot,Jump Shot,332,21000003,33.8983,-127,146,-118.3968,3,0,...,home,HOU,16449,493,0.39,1,8,0.36,10,0.326633
4,Jump Shot,Jump Shot,322,21000003,33.9623,-106,82,-118.3758,3,0,...,home,HOU,16448,546,0.39,0,6,0.30,10,0.326633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,Jump Shot,Jump Shot,468,21001230,33.9943,-98,50,-118.3678,4,0,...,away,SAC,18067,89,0.42,0,22,0.69,4,0.326633
1356,Jump Shot,Jump Shot,475,21001230,33.9553,144,89,-118.1258,4,0,...,away,SAC,18068,57,0.42,0,22,0.65,4,0.326633
1357,Jump Shot,Jump Shot,480,21001230,33.9963,-119,48,-118.3888,4,0,...,away,SAC,18069,15,0.42,0,22,0.61,4,0.326633
1358,Jump Shot,Jump Shot,506,21001230,33.9153,92,129,-118.1778,5,0,...,away,SAC,18071,244,0.42,1,25,0.60,4,0.326633


## Fit Linear Regression

ValueError: could not convert string to float: 'Driving Layup Shot'